In [13]:
# This script is used to show source img, target img, generated mask, gt mask of specific experiment
# input: output folder path, base experiment, list of experiments to compare
# intermidiate input: dataset, image and classes to visualize
# output: show source img, target img, heatmap, mask, gt in this ipynb

In [14]:
# import package
from pathlib import Path
import json
from omegaconf import OmegaConf
from utils.datasets import SegDataset
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [15]:
# input
output_folder = "./output"
# base experiment is experiments[0]
experiments = [
    "ddsseg_10_car_0_xxl_rev_delay",
    "ddsseg_10_car_0.5_xxl_rev_delay",
    "ddsseg_10_car_0.5_xxl_rev_delay_adj",
    "ddsseg_10_car_0.5_xxl_rev_delay_automobile",
    "ddsseg_10_car_0.5_xxl_rev_delay_sedan",
]
# use nicknames to show in the plot
exp_nickname = [
    "baseline",
    "0.5",
    "0.5_adj",
    "0.5_automobile",
    "0.5_sedan",
]

## Choosing Dataset, Image, ClassNames to Visualize

In [ ]:
# find available datasets and images' indexes
exp_path = [Path(output_folder) / exp for exp in experiments]
datasets_to_img_idx = {}
for dataset in exp_path[0].iterdir():
    metrics_json = dataset / "segmentation_metrics.json"
    with open(metrics_json, "r") as f:
        metrics = json.load(f)
        datasets_to_img_idx[dataset.name] = metrics["background"]["count"]
print("Available datasets and images count:")
for dataset in datasets_to_img_idx:
    print(f"{dataset}: {datasets_to_img_idx[dataset]} images")
print("Please choose one dataset to visualize.")

In [17]:
# intermidiate input
dataset_variant = 'voc_10_car'

In [ ]:
# construct dataset
dataset_names = ["voc_sim", "coco_cap", "voc", "context", "coco"]
for name in dataset_names:
    if name in dataset_variant:
        variant = dataset_variant.replace(name, "")
        dataset_cfg = OmegaConf.load(f"./configs/dataset/{name}.yaml")
        dataset_cfg.data_name_list = f"./data/{name}/val_id{variant}.txt"
        dataset = SegDataset(dataset_cfg)
        break
print(f"Available images in {dataset_variant}: {len(dataset)}")
print('Please choose one image to visualize.')

In [19]:
# intermidiate input
img_idx = 0

In [ ]:
# find available classes of the image
name, img_path, gt_path, label = dataset[img_idx]
cls_names = []
cls_name_to_idx = {}
for idx, cls_name in enumerate(dataset.category.keys()):
    cls_name_to_idx[cls_name] = idx + 1
    if label[idx] == 1:
        cls_names.append(cls_name)
print(f"Available classes of the image: {cls_names}")
print("Please choose classes to visualize.")

In [22]:
# intermidiate input
cls_to_visualize = ['car', 'chair', 'motorbike', 'people']

## Visualize Base Experiment Result of the Image

In [23]:
# load source image and gt to visualize
source_img = plt.imread(img_path)  # ndarray (H, W, C)
gt = np.array(Image.open(gt_path))  # ndarray (H, W)
if dataset.name == "coco":
    gt = gt + 1
    gt[gt > 80] = 0
cls_to_gt = {}
for cls in cls_to_visualize:
    cls_idx = cls_name_to_idx[cls]
    cls_gt = gt == cls_idx
    cls_to_gt[cls] = cls_gt.astype(np.uint8) * 255
# load target image, heatmaps, and masks to visualize
exp_cls_to_target = {}
exp_cls_to_heatmap = {}
exp_cls_to_mask = {}
for exp in exp_path:
    if exp.name not in exp_cls_to_target:
        exp_cls_to_target[exp.name] = {}
        exp_cls_to_heatmap[exp.name] = {}
        exp_cls_to_mask[exp.name] = {}
    mask_path = exp / dataset_variant / "masks" / f"{img_idx}.png"
    mask = np.array(Image.open(mask_path))  # ndarray (H, W)
    for cls in cls_to_visualize:
        target_path = exp / dataset_variant / "images" / f"{img_idx}_{cls}.png"
        heatmap_path = exp / dataset_variant / "heatmaps" / f"{img_idx}_{cls}.png"
        target = plt.imread(target_path)
        heatmap = plt.imread(heatmap_path)
        exp_cls_to_target[exp.name][cls] = target
        exp_cls_to_heatmap[exp.name][cls] = heatmap
        cls_mask = mask == cls_name_to_idx[cls]
        exp_cls_to_mask[exp.name][cls] = cls_mask.astype(np.uint8) * 255

In [ ]:
# different classes' heatmaps of the image in base experiment
# columns: source img, target img, heatmap, mask, gt
# rows: different classes of the image in base experiment
row_cnt = len(cls_to_visualize)
col_cnt = 5
fig, axs = plt.subplots(row_cnt, col_cnt, figsize=(col_cnt * 3, row_cnt * 3))

# Ensure axs is always a 2D array
if row_cnt == 1:
    axs = [axs]

for row, cls in enumerate(cls_to_visualize):
    axs[row][0].imshow(source_img)
    axs[row][1].imshow(exp_cls_to_target[experiments[0]][cls])
    axs[row][2].imshow(exp_cls_to_heatmap[experiments[0]][cls])
    axs[row][3].imshow(exp_cls_to_mask[experiments[0]][cls])
    axs[row][4].imshow(cls_to_gt[cls])

for i in range(row_cnt):
    for j in range(col_cnt):
        axs[i][j].tick_params(left=False, right=False, labelleft=False, labelbottom=False, bottom=False)

for j, title in enumerate(["source img", "target img", "heatmap", "mask", "gt"]):
    axs[0][j].set_title(title)

for i, cls in enumerate(cls_to_visualize):
    axs[i][0].set_ylabel(cls)

plt.tight_layout()
plt.show()

## Visualize Compared Experiment Results of the Image

In [ ]:
# different experiments' heatmaps of the image of the same class
# columns: source img, target img, heatmap, mask, gt
# rows: different experiments
for cls in cls_to_visualize:
    print(f"Class: {cls}")
    row_cnt = len(experiments)
    col_cnt = 5
    fig, axs = plt.subplots(row_cnt, col_cnt, figsize=(col_cnt * 3, row_cnt * 3))

    # Ensure axs is always a 2D array
    if row_cnt == 1:
        axs = [axs]

    for row, exp in enumerate(experiments):
        axs[row][0].imshow(source_img)
        axs[row][1].imshow(exp_cls_to_target[exp][cls])
        axs[row][2].imshow(exp_cls_to_heatmap[exp][cls])
        axs[row][3].imshow(exp_cls_to_mask[exp][cls])
        axs[row][4].imshow(cls_to_gt[cls])

    for i in range(row_cnt):
        for j in range(col_cnt):
            axs[i][j].tick_params(left=False, right=False, labelleft=False, labelbottom=False, bottom=False)

    for j, title in enumerate(["source img", "target img", "heatmap", "mask", "gt"]):
        axs[0][j].set_title(title)

    for i, exp in enumerate(exp_nickname):
        axs[i][0].set_ylabel(exp)

    plt.tight_layout()
    plt.show()